In [4]:

from docuverse.engines.data_template import DataTemplate
from docuverse.utils import read_config_file

In [33]:
file1="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.cc.json"
file2="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.dd.json"
data_template="../config/ibm_search_beir_data.yml"

data_config = read_config_file(data_template)

query_template = DataTemplate(data_config['query_format'])

In [34]:
from docuverse import SearchEngine

res1 = SearchEngine.read_output_(file1, query_template)
res2 = SearchEngine.read_output_(file2, query_template)

In [35]:
num_examples = len(res1)
top_k = 5
res = []
for i, (r1, r2) in enumerate(zip(res1, res2)):
    mismatch = False
    mismatched = []
    for j in range(top_k):
        if r1[j]['id'] != r2[j]['id']:
            mismatch = True
            #print(f"Mismatched at question {i}: {r1.question}, result {j}: {r1[j]} vs {r2[j]}")
            mismatched.append([[r1[j]['id'], r1[j]['score']], [r2[j]['id'], r2[j]['score']]])
    res.append(mismatched)

In [42]:
print(f"r1={res1[0][0]['text']}"
      f"\n\nr2={res2[0][0]['text']}")

r1=Enabling HyperSwap for a session
This feature is only available after APAR OA53082 is applied to the z/OS system. Unbox Secondary Volumes Select this option to have the load of the configuration automatically unbox all secondary volumes in the configuration. When this option is not selected, a load of the configuration fails when secondary volumes are in a boxed state. This feature is only available after APAR OA53082 is applied to the z/OS system. On Planned HyperSwap Error: Partition out the failing system(s) and continue swap processing on the remaining system(s) Select this option to partition out the failing system and continue the swap processing on any remaining systems. Disable HyperSwap after attempting backout Select this option to enable IOS to back out the HyperSwap operation, if possible, if an error occurs during HyperSwap processing. HyperSwap is disabled. On Unplanned HyperSwap Error: Partition out the failing system(s) and continue swap processing on the remaining s